In [10]:

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import numpy as np
import os
from args import get_parser
import pickle
from model import get_model
from torchvision import transforms
from utils.output_utils import prepare_output
from PIL import Image
import time

In [12]:
from modules.encoder import EncoderVisionTransformer

In [2]:
# code will run in gpu if available and if the flag is set to True, else it will run on cpu
use_gpu = False
device = torch.device('cuda' if torch.cuda.is_available() and use_gpu else 'cpu')
map_loc = None if torch.cuda.is_available() and use_gpu else 'cpu'

In [3]:
ingrs_vocab = pickle.load(open('/data/prateek/github/see-food/garbage/ingr_vocab.pkl', 'rb'))
vocab = pickle.load(open('/data/prateek/github/see-food/garbage/instr_vocab.pkl', 'rb'))


ingr_vocab_size = len(ingrs_vocab)
instrs_vocab_size = len(vocab)
output_dim = instrs_vocab_size

In [4]:
print(instrs_vocab_size, ingr_vocab_size)

4660 512


In [26]:
t = time.time()
import sys; sys.argv=['']; del sys
args = get_parser()
args.maxseqlen = 15

# args.ingrs_only=False
args.use_vision_transformer=True

In [27]:
model = get_model(args, ingr_vocab_size, instrs_vocab_size)

using vision: True


Some weights of the model checkpoint at google/vit-base-patch16-224 were not used when initializing ViTModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/nas/home/pchhikar/anaconda3/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_av

In [28]:
for name, module in model.named_modules():
    print(name, type(module))

 <class 'model.InverseCookingModel'>
ingredient_encoder <class 'modules.encoder.EncoderLabels'>
ingredient_encoder.linear <class 'torch.nn.modules.sparse.Embedding'>
recipe_decoder <class 'modules.transformer_decoder.DecoderTransformer'>
recipe_decoder.embed_tokens <class 'torch.nn.modules.sparse.Embedding'>
recipe_decoder.embed_positions <class 'modules.transformer_decoder.LearnedPositionalEmbedding'>
recipe_decoder.layers <class 'torch.nn.modules.container.ModuleList'>
recipe_decoder.layers.0 <class 'modules.transformer_decoder.TransformerDecoderLayer'>
recipe_decoder.layers.0.self_attn <class 'modules.multihead_attention.MultiheadAttention'>
recipe_decoder.layers.0.self_attn.out_proj <class 'torch.nn.modules.linear.Linear'>
recipe_decoder.layers.0.cond_att <class 'modules.multihead_attention.MultiheadAttention'>
recipe_decoder.layers.0.cond_att.out_proj <class 'torch.nn.modules.linear.Linear'>
recipe_decoder.layers.0.fc1 <class 'torch.nn.modules.linear.Linear'>
recipe_decoder.layers

In [13]:
# model.image_encoder = EncoderVisionTransformer(args.embed_size)

Some weights of the model checkpoint at google/vit-base-patch16-224 were not used when initializing ViTModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# m

In [29]:
vit_model_path = '/data/prateek/github/see-food/checkpoints_10k/inversecooking/ingr_only_vit/checkpoints/modelbest.ckpt'
partial_vit_state_dict = torch.load(vit_model_path, map_location=torch.device('cpu'))

for keys in partial_vit_state_dict.keys():
    print(keys)


ingredient_encoder.linear.weight
recipe_decoder.embed_tokens.weight
recipe_decoder.embed_positions.weight
recipe_decoder.layers.0.self_attn.in_proj_weight
recipe_decoder.layers.0.self_attn.in_proj_bias
recipe_decoder.layers.0.self_attn.out_proj.weight
recipe_decoder.layers.0.self_attn.out_proj.bias
recipe_decoder.layers.0.cond_att.in_proj_weight
recipe_decoder.layers.0.cond_att.in_proj_bias
recipe_decoder.layers.0.cond_att.out_proj.weight
recipe_decoder.layers.0.cond_att.out_proj.bias
recipe_decoder.layers.0.fc1.weight
recipe_decoder.layers.0.fc1.bias
recipe_decoder.layers.0.fc2.weight
recipe_decoder.layers.0.fc2.bias
recipe_decoder.layers.0.layer_norms.0.weight
recipe_decoder.layers.0.layer_norms.0.bias
recipe_decoder.layers.0.layer_norms.1.weight
recipe_decoder.layers.0.layer_norms.1.bias
recipe_decoder.layers.0.layer_norms.2.weight
recipe_decoder.layers.0.layer_norms.2.bias
recipe_decoder.layers.1.self_attn.in_proj_weight
recipe_decoder.layers.1.self_attn.in_proj_bias
recipe_decoder

In [21]:
model.image_encoder.load_state_dict(partial_vit_state_dict)

RuntimeError: Error(s) in loading state_dict for EncoderVisionTransformer:
	Missing key(s) in state_dict: "vit.embeddings.cls_token", "vit.embeddings.position_embeddings", "vit.embeddings.patch_embeddings.projection.weight", "vit.embeddings.patch_embeddings.projection.bias", "vit.encoder.layer.0.attention.attention.query.weight", "vit.encoder.layer.0.attention.attention.query.bias", "vit.encoder.layer.0.attention.attention.key.weight", "vit.encoder.layer.0.attention.attention.key.bias", "vit.encoder.layer.0.attention.attention.value.weight", "vit.encoder.layer.0.attention.attention.value.bias", "vit.encoder.layer.0.attention.output.dense.weight", "vit.encoder.layer.0.attention.output.dense.bias", "vit.encoder.layer.0.intermediate.dense.weight", "vit.encoder.layer.0.intermediate.dense.bias", "vit.encoder.layer.0.output.dense.weight", "vit.encoder.layer.0.output.dense.bias", "vit.encoder.layer.0.layernorm_before.weight", "vit.encoder.layer.0.layernorm_before.bias", "vit.encoder.layer.0.l

In [14]:
recipe_model_path = '/data/prateek/github/see-food/paper_weights/modelbest.ckpt'
partial_recipe_state_dict = torch.load(recipe_model_path, map_location=torch.device('cpu'))
print(partial_recipe_state_dict.keys())

odict_keys(['ingredient_encoder.linear.weight', 'recipe_decoder.embed_tokens.weight', 'recipe_decoder.embed_positions.weight', 'recipe_decoder.layers.0.self_attn.in_proj_weight', 'recipe_decoder.layers.0.self_attn.in_proj_bias', 'recipe_decoder.layers.0.self_attn.out_proj.weight', 'recipe_decoder.layers.0.self_attn.out_proj.bias', 'recipe_decoder.layers.0.cond_att.in_proj_weight', 'recipe_decoder.layers.0.cond_att.in_proj_bias', 'recipe_decoder.layers.0.cond_att.out_proj.weight', 'recipe_decoder.layers.0.cond_att.out_proj.bias', 'recipe_decoder.layers.0.fc1.weight', 'recipe_decoder.layers.0.fc1.bias', 'recipe_decoder.layers.0.fc2.weight', 'recipe_decoder.layers.0.fc2.bias', 'recipe_decoder.layers.0.layer_norms.0.weight', 'recipe_decoder.layers.0.layer_norms.0.bias', 'recipe_decoder.layers.0.layer_norms.1.weight', 'recipe_decoder.layers.0.layer_norms.1.bias', 'recipe_decoder.layers.0.layer_norms.2.weight', 'recipe_decoder.layers.0.layer_norms.2.bias', 'recipe_decoder.layers.1.self_attn.

In [19]:
model.image_encoder

EncoderVisionTransformer(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0): ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_f

In [ ]:
layer1_state_dict = {'0.weight': partial_state_dict['0.weight'],
                     '0.bias': partial_state_dict['0.bias']}